### Randomforest

- **주요 하이퍼파라미터**
    - n_estimators
        - DecisionTree 모델의 개수
        - 학습할 시간과 메모리가 허용하는 범위에서 클수록 좋다. 
    - max_features
        - 각 트리에서 선택할 feature의 개수
        - 클수록 각 트리간의 feature 차이가 크고 작을 수록 차이가 적게 나게 된다.
    - DecisionTree의 하이퍼파라미터들
        - Tree의 최대 깊이, 가지를 치기 위한 최소 샘플 수 등 Decision Tree에서 과적합을 막기 위한 파라미터들을 랜덤 포레스트에 적용할 수 있다.

| 알고리즘 | 유형 | 주요 특징 | 주요 하이퍼파라미터 |
|-----------|--------|------------|-------------------------|
| **Random Forest** | 분류/회귀 | 다수의 결정트리 앙상블, 과적합 완화, 특성 중요도 제공 | `n_estimators`, `max_depth`, `max_features`, `bootstrap` |


### 

#### 1. 데이터 준비 및 X,y 생성

In [ ]:
import pandas as pd
wine = pd.read_csv("data/wine.csv")

X = wine.drop(columns="color")#.values
y = wine['color']#.values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(['A', 'B', 'C'])
X['quality'] = le.fit_transform(X['quality']) 

#### 2. 학습/검증용 데이터 분할 (Train/Test Split)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

#### 3. 전처리 및 모델 pipeline 만들기 <br> 4. 하이퍼파라미터 탐색 및 모델 튜닝 (GridSearchCV)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
params = {
    "max_depth":range(1, 14), 
    "max_leaf_nodes": range(10, 56),
    "min_samples_leaf": range(10, 1001, 50),
    "max_features": range(1, 13)
}
# gs = GridSearchCV(
gs = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=0),
    params,
    n_iter=60,    # 테스트할 하이퍼파라미터 조합 개수 (random하게 조합을 선택한다.)
    scoring='accuracy', 
    cv=5,         # cross validation fold 개수
    n_jobs=-1
)

#### 5. 모델 학습 (Fit)

In [ ]:
gs.fit(X_train, y_train)

#### 6. 학습 결과 평가 (Best Params, Score)

In [ ]:
gs.best_score_
gs.best_params_

In [ ]:
import pandas as pd
result_cv = pd.DataFrame(gs.cv_results_).sort_values("rank_test_score")
result_cv.head()

#### 7. 테스트셋 최종 평가 (Best Estimator Predict)

gs.best_estimator_  Feature importance 확인

In [ ]:
best_model = gs.best_estimator_
fi = pd.Series(best_model.feature_importances_, index=wine.columns[:-1]).sort_values(ascending=False)

In [ ]:
# 시각화

from sklearn.tree import export_graphviz
from graphviz import Source

graph = Source(
    export_graphviz(
        best_model, 
        feature_names=wine.columns[:-1],
        class_names = ["White", "Red"],
        filled=True,
        rounded=True
    )
)
graph

### 회귀
- DecisionTreeRegressor  사용

#### 1. 데이터 준비 및 X,y 생성

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/boston_dataset.csv")
X = df.drop(columns='MEDV')
y = df['MEDV']


#### 2. 학습/검증용 데이터 분할 (Train/Test Split)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)    
X_train.shape, X_test.shape

#### 3. 전처리 및 모델 pipeline 만들기 <br> 4. 하이퍼파라미터 탐색 및 모델 튜닝 (GridSearchCV)

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from graphviz import Source
from metrics import print_regression_metrcis

model = DecisionTreeRegressor(max_depth = 2, random_state=0)  # 2개의 단계만 내려가도록 max_depth 설정

#### 5. 모델 학습 (Fit)

In [ ]:
model.fit(X_train, y_train)

#### 6. 학습 결과 평가 (Best Params, Score)

In [ ]:
model.get_depth()
model.get_n_leaves()

#### 7. 테스트셋 최종 평가 (Best Estimator Predict)

gs.best_estimator_  Feature importance 확인

In [ ]:
print_regression_metrcis(y_train, model.predict(X_train))
print_regression_metrcis(y_test, model.predict(X_test))

In [ ]:
## 분기 구조를 시각화
graph = Source(
    export_graphviz(
        model, 
        feature_names=X.columns, 
        filled=True, rounded=True
    )
)

graph


# value = 값의 평균
# squared_error = (각 실제 값 - 평균)**2
# 그래서 이 squared_error 가 최소화하기 위해 또 분기를 함.-> 그래서 분기를 하기 위한 첫번째 질문이 LSTAT <= 8.13인지를 물어봄.
graph

In [ ]:
fi = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
fi